<p style ="text-align:center">
    <img src="http://epecora.com.br/DataFiles/BannerUFPR.png" width="700" alt="PPGOLD/PPGMNE Python:INTRO"  />
</p>

# Data Science for Businnes - Aula 04

## Prof. Eduardo Pécora

## K-Fold Cross Validation
Tempo estimado: **30** minutos

## Objetivos

Após completar esta aula, você será capaz de:

* Separar os dados em dados de treino e teste
* Usar o K-Fold para avaliar a eficiência do modelo

## Bibliotecas

In [1]:
# importando a biblioteca pandas para manipulação de dados
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelBinarizer

# importando as bibliotecas do matplotlib para gerar gráficos
import matplotlib as mpl
import matplotlib.pyplot as plt

# Importando a biblioteca numpy e math que fornece funções matemáticas básicas
import numpy as np
import math 

# Importando biblioteca do seaborn para gerar gráficos mais atraentes e informativos
import seaborn as sns

# Importando a classe LinearRegression do sklearn 
# Essa classe implementa uma versão da regressão linear simples ou múltipla
# Usado para modelar a relação entre uma variável dependente contínua e uma ou mais variáveis independentes.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Usado para exibir os gráficos gerados pela biblioteca Matplotlib diretamente no notebook, sem precisar abrir uma janela externa.
%matplotlib inline


## Obtendo dados

In [2]:
# Caminho do arquivo csv
caminho = "https://raw.githubusercontent.com/EduPekUfpr/PythonProject/refs/heads/main/Dados/MeuAutoLimpo.csv" 

#Obtendo arquivo e passando-o para um dataframe
df = pd.read_csv(caminho) 
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,122,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,164,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


## Tranformação das variáveis

*Dúvidas? Veja a aula de Regressão Linear.*

In [3]:
df_dummy = df.copy()

label_binarizer = LabelBinarizer()
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html

dicotomica = df_dummy['num-of-doors']
label_binarizer.fit(dicotomica)
df_dummy['num-of-doors'] = label_binarizer.transform(dicotomica)

# renomeando as marcas
df_dummy['make'] = df_dummy['make'].replace({'bmw':'BMW', 'doge': 'dodge', 'VW':'volkswagen', 'volv1':'volvo' })

# Crie uma instância do codificador OneHotEncoder
encoder = OneHotEncoder()

# Ajuste e transforme os dados da coluna "make"
poly_array = encoder.fit_transform(df_dummy[['make']])

# Crie um DataFrame Pandas com as variáveis dummy
poly_df = pd.DataFrame(poly_array.toarray(), columns=encoder.get_feature_names_out(['make']))

# Concatene o DataFrame dummy com os outros dados
df_dummy = pd.concat([df_dummy.drop('make', axis=1), poly_df], axis=1)

df_dummy.drop(['fuel-type', 'aspiration', 'body-style', 'drive-wheels', 
               'engine-location', 'engine-type', 'num-of-cylinders', 'fuel-system'], axis =1, inplace = True)
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   symboling           201 non-null    int64  
 1   normalized-losses   201 non-null    int64  
 2   num-of-doors        201 non-null    int64  
 3   wheel-base          201 non-null    float64
 4   length              201 non-null    float64
 5   width               201 non-null    float64
 6   height              201 non-null    float64
 7   curb-weight         201 non-null    int64  
 8   engine-size         201 non-null    int64  
 9   bore                201 non-null    float64
 10  stroke              201 non-null    float64
 11  compression-ratio   201 non-null    float64
 12  horsepower          201 non-null    float64
 13  peak-rpm            201 non-null    float64
 14  city-mpg            201 non-null    int64  
 15  highway-mpg         201 non-null    int64  
 16  price   

## K-Fold CrossValidation

In [4]:
from sklearn.model_selection import KFold

# Original Data
X = df_dummy.drop(['price'], axis = 1)
Y = df_dummy['price']

# Separate features and target variable
X = df_dummy.drop(columns=['price'])
y = df_dummy['price']

# Initialize the KFold object with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

## Modelo Vazio

In [5]:
lm = LinearRegression()
lm

LinearRegression()

## Avalia o modelo nas variáveis de treino e teste

In [6]:
# Initialize lists to store the MSE for each fold
mse_scores = []
mae_scores = []
mape_scores = []
r2_scores = []

# Iterate over each fold
for train_index, test_index in kf.split(X):
    # Split data into train and test sets for this fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    # Train a linear regression model
    lm.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = lm.predict(X_test)
    
    # Calculate Mean Squared Error for this fold
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape= mae/y_test.mean()
    
    mse_scores.append(mse)
    mae_scores.append(mae)
    mape_scores.append(mape)
    r2_scores.append(lm.score(X_test,y_test))
    

# Calculate the average MSE across all folds
print("Average MSE :", np.mean(mse_scores))
print("Average MAE :", np.mean(mae_scores))
print("Average MAPE:", np.mean(mape_scores))
print("Average R2  :", np.mean(r2_scores))

print(r2_scores)

Average MSE : 5981995.149120822
Average MAE : 1667.0680904865349
Average MAPE: 0.12721135699888847
Average R2  : 0.8489048978906762
[0.9311309125873716, 0.9475035248429298, 0.7356466398391832, 0.7843334316705413, 0.880536308320907, 0.9472403357467198, 0.8164383439470402, 0.863057479592229, 0.7148255476071467, 0.8683364547526953]


## Um código mais compacto

In [7]:
from sklearn.model_selection import cross_val_score

splits = 15
scores = cross_val_score(lm, df_dummy.drop(['price'],axis=1), df_dummy['price'], cv = splits, scoring = 'r2')
print(scores)

[ -2.80750069   0.8396646   -1.71378382   0.91649639   0.32483703
   0.9163264  -16.72048923  -1.34087504   0.32226062   0.68977377
   0.4121155  -11.4513833    0.3569937   -0.30238093   0.33779679]


* O R² negativo ocorre porque as previsões do modelo estão tão distantes dos valores reais que isso aumenta a soma residual dos quadrados (erros), resultando em um ajuste pior do que simplesmente prever a média.
* R² < -1 pode acontecer quando os erros das previsões do modelo são ainda maiores do que a variância total dos dados.


$R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2}$


## Leave One Out

In [8]:
from sklearn.model_selection import LeaveOneOut, cross_val_predict

loo = LeaveOneOut()

predictions = cross_val_predict(lm, df_dummy.drop(['price'],axis=1), df_dummy['price'], cv = loo)

mae = mean_absolute_error(df_dummy['price'], predictions)

print("MAE :", mae)

MAE : 1706.7701114908346


## Fique Conectado

- [![YouTube](https://img.icons8.com/ios-filled/40/000000/youtube-play.png)](https://www.youtube.com/@LigaDataScience/videos)  
  Explore nossos vídeos educacionais e webinars sobre ciência de dados, machine learning e inteligência artificial. Inscreva-se para não perder nenhuma atualização!

- [![LinkedIn](https://img.icons8.com/ios-filled/40/000000/linkedin.png)](https://www.linkedin.com/company/liga-data-science-ufpr/)  
  Siga-nos no LinkedIn para as últimas novidades, oportunidades de carreira e networking profissional no campo da ciência de dados.

- [![Instagram](https://img.icons8.com/ios-filled/40/000000/instagram-new.png)](https://www.instagram.com/ligadatascience/)  
  Confira nosso Instagram para conteúdos dos bastidores, destaques de eventos e o dia a dia da Liga Data Science. Faça parte da nossa jornada!


## Autores

<a href="https://www.linkedin.com/in/eduardopecora/" target="_blank">Eduardo Pecora</a>

## Referências:

* Documentação da biblioteca <a href="https://pandas.pydata.org/docs/">Pandas</a>
* Documentação do método <a href=https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html>cross_val_score</a>
* Documentação do método <a href=https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.KFold.html>KFold</a>
* Documentação do método <a href=https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html>train_test_split</a>


## Log de modificações

| Data | Versão | Modificado por | Descrição |
| -----------| ------- | ---------- | ---------------------------------- |
| 24-04-2024       | 1.0     | Eduardo Pecora    | Inicial               |